# Constraints

- Time limit: integer 0<= T <= 999
- Number of bunnies: integer 0? <= B <= 5

In [3]:
class Graph:
    def __init__(self, matrix):
        self.cost_matrix = matrix
        self.n = len(matrix)
        self.n_rabbits = self.n - 2
        self.edges = [(i, j, self.cost_matrix[i][j]) for i in range(self.n) for j in range(self.n)]

    def any_self_distance_negative(self):
        for i in range(self.n):
            if self.cost_matrix[i][i] < 0:
                return True
        return False

    # Bellman-Ford algorithm
    def BellmanFord(self, start_node):
        distance = [float("inf")] * self.n
        distance[start_node] = 0 # Even if the path to go to itself is not zero, you can just stay put

        for _ in range(len(self.edges) - 1):
            for (u, v, weight) in self.edges:
                if distance[u] + weight < distance[v]:
                    distance[v] = distance[u] + weight

        # check for negative-weight cycles
        for (u, v, weight) in self.edges:
            if distance[u] + weight < distance[v]:
                return "Negative cycle"

        return [int(d) for d in distance]

In [4]:
# depth first search
def possible_paths(graph, distances, path_list, cumulative_time_to_path, cost_limit):
    """
    Returns a list of paths that are valid starting from the given path with the given cumulative cost, and that end in the last node.
    """
    if path_list[-1] == graph.n - 1:
        # For homogeneity of return type (list of paths)
        return [path_list]
    
    list_possible_nodes = []
    
    # Similar idea to A*, I guess. We need to take into account the distance to the end node
    for possible_node in range(graph.n):
        if possible_node not in path_list and cumulative_time_to_path + distances[path_list[-1]][possible_node] + distances[possible_node][-1] <= cost_limit:
            list_possible_nodes.append(possible_node)
    
    pos_paths = [path_list + [possible_node] for possible_node in list_possible_nodes]
    possible_cumulative_times = [cumulative_time_to_path + distances[path_list[-1]][possible_node] for possible_node in list_possible_nodes]

    ret = [possible_paths(graph, distances, path, cum_path_cost, cost_limit) for path, cum_path_cost in zip(pos_paths, possible_cumulative_times)]
    # unnest once
    return [item for sublist in ret for item in sublist]

In [5]:
def max_score_paths(graph, time_limit):
    """
    Returns a list with the paths with the maximum number of bunnies that can be saved in the given time limit.
    """
    if graph.any_self_distance_negative():
        # all vertices can be reached as long as you go back in time long enough
        return "All"
    
    distances = [graph.BellmanFord(i) for i in range(graph.n)]

    if any([d == "Negative cycle" for d in distances]):
        return "All" # I would much rather use Results like in Rust than keep using flag return values for this, yes

    cumulative_time_of_path = 0
    path = [0]
    paths = possible_paths(graph, distances, path, cumulative_time_of_path, time_limit)
    
    paths = [p for p in paths if p[-1] == graph.n - 1]
    
    def score(path):
        """Counts the number of rabbits that are visited."""
        return sum([1 for i in range(1, graph.n - 1) if i in path])


    paths_score = [score(path) for path in paths]
    max_score_paths = tuple(path for path, score in zip(paths, paths_score) if score == max(paths_score))
    
    return max_score_paths

In [6]:
def solution(times, time_limit):
    graph = Graph(times)
    paths = max_score_paths(graph, time_limit)
    if paths == "All":
        return [i for i in range(graph.n_rabbits)]
    bunny_list_of_each_path = [[index - 1 for index in path[1:-1]] for path in paths]
    bunny_list_of_each_path = [set(tuple(bunny_list)) for bunny_list in bunny_list_of_each_path]

    definitive_list = []
    for bunny in range(graph.n_rabbits):
        if any([bunny in path for path in bunny_list_of_each_path]):
            definitive_list.append(bunny)
            bunny_list_of_each_path = [path for path in bunny_list_of_each_path if bunny in path]
    return definitive_list

In [7]:
# testing
print(solution([[0, 2, 2, 2, -1], [9, 0, 2, 2, -1], [9, 3, 0, 2, -1], [9, 3, 2, 0, -1], [9, 3, 2, 2, 0]], 1)) # [1,2]
print(solution([[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 3)    ) # [0,1]

[1, 2]
[0, 1]
